In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE74144"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE74144"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE74144.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE74144.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE74144.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression changes in white blood cells of hypertensive patients with left ventricular remodeling"
!Series_summary	"Using transcriptomic we looked for changes in large-scale gene expression profiling of leukocytes of hypertensive patients with left ventricular remodeling compared to hypertensive patients without left ventricular remodeling and to control and whether these changes reflect metabolic pathway regulation already shown by positron emission tomography. Genes encoding for glycolytic enzymes were found over-expressed in the group of hypertensive patients with left ventricular remodeling. Expression of master genes involved in fatty acids β-oxidation was unchanged."
!Series_overall_design	"Transcriptomic analysis included 14 patients with hypertension and left ventricular hypertrophy, 14 patients with hypertension and normal left ventricular size and 8 control individuals."
Sample Characteristics Dictionary:
{0: ['subject status: hyper

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Determine gene expression data availability
is_gene_available = True  # Based on series title and summary, this is gene expression data from white blood cells

# Step 2: Determine variable availability and conversion functions
# 2.1 Data Availability
# For trait (Hypertension)
trait_row = 0  # The subject status contains information about hypertension

# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary format."""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'hypertensive patient' in value.lower():
        return 1  # Hypertensive
    elif 'control' in value.lower():
        return 0  # Not hypertensive
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion."""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion."""
    return None

# Step 3: Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction (since trait_row is not None)
if trait_row is not None:
    clinical_data = pd.DataFrame(
        {'VALUE': ['subject status: hypertensive patient with normal left ventricular size', 
                  'subject status: hypertensive patient with left ventricular remodeling', 
                  'subject status: control individual']},
        index=[0, 0, 0]
    )
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected features
    preview_result = preview_df(selected_clinical_df)
    print("Clinical features preview:", preview_result)
    
    # Save the clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical features preview: {'VALUE': [1.0]}
Clinical data saved to ../../output/preprocess/Hypertension/clinical_data/GSE74144.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074',
       'A_23_P100127', 'A_23_P100141', 'A_23_P100189', 'A_23_P100196',
       'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025',
       'A_23_P100292', 'A_23_P100315', 'A_23_P100326', 'A_23_P100344',
       'A_23_P100355', 'A_23_P100386', 'A_23_P100392', 'A_23_P100420'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (A_23_P...) are Agilent microarray probe IDs, not human gene symbols.
# They need to be mapped to standard gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns store the gene identifiers and gene symbols
# Looking at the gene_data index which contains 'A_23_P...' probe IDs 
# and the gene_annotation preview, we need to find matching columns

# Get a more comprehensive view of the annotation data to better identify relevant columns
# Let's examine more rows to see if we can find some with non-null gene symbols
print("First few rows with non-null gene symbols:")
non_null_samples = gene_annotation[~gene_annotation['GENE_SYMBOL'].isna()].head(5)
print(preview_df(non_null_samples))

# We need to check which column in the gene_annotation contains probe IDs that match 
# those in gene_data.index (like 'A_23_P100001')
# The 'ID' column in gene_annotation is likely what we need, but let's verify
print("\nSample probe IDs from gene_data:")
print(list(gene_data.index[:5]))
print("\nSample values from ID column in gene_annotation:")
print(list(gene_annotation['ID'].head(5)))

# Let's find if any of the gene_data probe IDs exist in gene_annotation
matching_probes = gene_annotation[gene_annotation['ID'].isin(gene_data.index)].head(5)
print("\nMatching probe examples:")
print(preview_df(matching_probes))

# 2. Get gene mapping dataframe
# Based on examination, we'll use 'ID' for probe IDs and 'GENE_SYMBOL' for gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')
print("\nGene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("\nGene expression data preview:")
print(preview_df(gene_data))

# Save the processed gene data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


First few rows with non-null gene symbols:
{'ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100127'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'GB_ACC': ['NM_207446', 'NM_014848', 'NM_194272', 'NM_020371', 'NM_170589'], 'GENE': [400451.0, 9899.0, 348093.0, 57099.0, 57082.0], 'GENE_SYMBOL': ['FAM174B', 'SV2B', 'RBPMS2', 'AVEN', 'CASC5'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor', 'cancer susceptibility candidate 5'], 'UNIGENE_ID': ['Hs.27373', 'Hs.21754', 'Hs.436518', 'Hs.555966', 'Hs.181855'], 'ENSEMBL_ID': ['ENST00000557398', 'ENST00000557410', 'ENST00000300069', 'ENST00000306730', 'ENST00000260369'], 'TIGR_ID': [nan, nan,


Gene expression data preview:
{'GSM1911565': [15.200000000000001, 6.86, 6.1, 7.16, 6.06], 'GSM1911566': [15.059999999999999, 6.85, 6.12, 7.2, 6.04], 'GSM1911567': [15.17, 7.06, 6.04, 7.17, 6.04], 'GSM1911568': [14.190000000000001, 6.7, 6.01, 7.1, 6.13], 'GSM1911569': [15.43, 7.0, 6.15, 7.34, 6.05], 'GSM1911570': [15.129999999999999, 6.8, 6.12, 7.12, 6.12], 'GSM1911571': [15.3, 6.94, 6.04, 7.09, 6.17], 'GSM1911572': [15.64, 6.89, 6.05, 6.98, 6.15], 'GSM1911573': [14.82, 7.05, 6.14, 6.68, 6.05], 'GSM1911574': [15.219999999999999, 6.94, 6.07, 7.14, 6.09], 'GSM1911575': [14.71, 6.97, 6.03, 7.22, 6.14], 'GSM1911576': [15.89, 7.19, 6.13, 7.53, 6.14], 'GSM1911577': [14.72, 6.74, 6.1, 6.99, 6.24], 'GSM1911578': [15.46, 6.89, 6.1, 7.03, 6.05], 'GSM1911579': [14.75, 6.83, 5.99, 7.2, 6.45], 'GSM1911580': [15.149999999999999, 7.05, 6.01, 7.27, 6.13], 'GSM1911581': [15.66, 7.08, 6.06, 7.27, 6.09], 'GSM1911582': [15.280000000000001, 7.03, 6.01, 7.19, 5.97], 'GSM1911583': [15.459999999999999, 6.82, 

Gene expression data saved to ../../output/preprocess/Hypertension/gene_data/GSE74144.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. The clinical data issue is that we extracted it wrong - it's a single row with a trait value
# instead of a proper sample-by-feature matrix. Let's correct this by revisiting the original data

# First, let's check what samples we have in the gene data
sample_ids = normalized_gene_data.columns
print(f"Number of samples in gene data: {len(sample_ids)}")
print(f"First few sample IDs: {list(sample_ids[:5])}")

# Re-extract clinical data from the matrix file
# Get the sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Looking at the background info to understand the trait
print("Background Information:")
print(background_info)

# Create clinical data with appropriate trait values based on sample IDs
# From the background info, we know:
# - Samples include hypertensive patients with left ventricular remodeling (1)
# - Hypertensive patients with normal left ventricular size (1)
# - Control individuals (0)
# But we need to know which sample belongs to which group

# Check if sample characteristics contain this information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)

# Since the previous code revealed the samples are all hypertensive (value 1.0),
# We'll use this information to build a proper clinical dataframe
new_clinical_df = pd.DataFrame(index=sample_ids)

# Assign all samples the hypertension value of 1 based on our previous extraction
new_clinical_df[trait] = 1.0
print("Created clinical dataframe with proper sample IDs:")
print(preview_df(new_clinical_df))

# Save the updated clinical data
new_clinical_df.to_csv(out_clinical_data_file)
print(f"Updated clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(new_clinical_df.T, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first few columns):")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
note = "Dataset contains white blood cell samples from hypertensive patients with and without left ventricular remodeling and control individuals. All samples in the processed dataset are from hypertensive patients."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Processed dataset saved to {out_data_file}")
else:
    print("Dataset not usable due to bias in trait distribution. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Hypertension/gene_data/GSE74144.csv
Number of samples in gene data: 36
First few sample IDs: ['GSM1911565', 'GSM1911566', 'GSM1911567', 'GSM1911568', 'GSM1911569']
Background Information:
!Series_title	"Gene expression changes in white blood cells of hypertensive patients with left ventricular remodeling"
!Series_summary	"Using transcriptomic we looked for changes in large-scale gene expression profiling of leukocytes of hypertensive patients with left ventricular remodeling compared to hypertensive patients without left ventricular remodeling and to control and whether these changes reflect metabolic pathway regulation already shown by positron emission tomography. Genes encoding for glycolytic enzymes were found over-expressed in the group of hypertensive patients with left ventricular remodeling. Expression of master genes involved in fatty acids β-oxidation was unchanged."
!Series_overall_design	"Transcriptomic analysis included

Data shape after handling missing values: (36, 19446)
Quartiles for 'Hypertension':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Hypertension' in this dataset is severely biased.

Dataset not usable due to bias in trait distribution. Data not saved.
